In [1]:
%load_ext autoreload
%autoreload 2
from finetune import *

/Users/chaosarium/anaconda3/envs/multi/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
args

{'cultural_corpus': 'yo-bm25-10000',
 'pretrained_model': 'distilbert-base-uncased',
 'corpus_chunk_size': 128,
 'wwm_probability': 0.15,
 'batch_size': 64}

In [3]:
tokenizer = mk_tokenizer(args)
mlm_model, mc_model = mk_models(args)

mask token id is 103 and mask token is [MASK]
tokenizer max len 512


Some weights of the model checkpoint at FacebookAI/xlm-roberta-base were not used when initializing XLMRobertaForMaskedLM: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing XLMRobertaForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLMRobertaForMultipleChoice were not initialized from the model checkpoint at FacebookAI/xlm-roberta-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


mlm_model num param: 278295186
mc_model num param: 278044417


In [4]:
fill_mask(args, mlm_model, tokenizer, f"The cat said {tokenizer.decode(tokenizer.mask_token_id)}.")

input: The cat said [MASK].
pred 0: The cat said _[MASK]_.
pred 1: The cat said _[unused12]_.
pred 2: The cat said _[unused255]_.
pred 3: The cat said _georgia_.
pred 4: The cat said _[unused213]_.
pred 5: The cat said _[unused192]_.
pred 6: The cat said _[unused58]_.
pred 7: The cat said _[unused441]_.
pred 8: The cat said _ы_.
pred 9: The cat said _passenger_.


In [5]:
toy_text = f"The cat said {tokenizer.decode(tokenizer.mask_token_id)}."
toy_input = tokenizer(toy_text, return_tensors="pt")
mlm_model(**toy_input).keys()

odict_keys(['logits'])

In [7]:
lm_corpus, corpus_mask_collator, corpus_wwm_collator = mk_corpus(args, tokenizer)
lm_corpus

Map: 100%|██████████| 30/30 [00:00<00:00, 10006.29 examples/s]

index 0 train example:
{'input_ids': [101, 19204, 17629, 2015, 2024, 2061, 2033, 5004, 102, 101, 1996, 4937, 2056, 2033, 5004, 102, 101, 19204, 17629, 2015, 2024, 2061, 2033, 5004, 102, 101, 8870, 2360, 2033, 5004, 102, 101, 19204, 17629, 2015, 2024, 2061, 2033, 5004, 102, 101, 8870, 2360, 2033, 5004, 102, 101, 1996, 4937, 2056, 2033, 5004, 102, 101, 19204, 17629, 2015, 2024, 2061, 2033, 5004, 102, 101, 8870, 2360, 2033, 5004, 102, 101, 1996, 4937, 2056, 2033, 5004, 102, 101, 1996, 4937, 2056, 2033, 5004, 102, 101, 19204, 17629, 2015, 2024, 2061, 2033, 5004, 102, 101, 8870, 2360, 2033, 5004, 102, 101, 19204, 17629, 2015, 2024, 2061, 2033, 5004, 102, 101, 19204, 17629, 2015, 2024, 2061, 2033, 5004, 102, 101, 19204, 17629, 2015, 2024, 2061, 2033, 5004, 102, 101, 8870, 2360, 2033], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'word_ids', 'labels'],
        num_rows: 15
    })
    val: Dataset({
        features: ['input_ids', 'attention_mask', 'word_ids', 'labels'],
        num_rows: 1
    })
})

## MLM train

In [8]:
training_args = TrainingArguments(
    output_dir=f"xlmr-finetuned",
    overwrite_output_dir=True,
    evaluation_strategy="epoch",
    learning_rate=5e-4,
    weight_decay=0.01,
    per_device_train_batch_size=args['batch_size'],
    per_device_eval_batch_size=args['batch_size'],
    logging_steps=1,
    num_train_epochs=10,
    use_cpu=True,
    use_mps_device=False,
)
trainer = Trainer(
    model=mlm_model,
    args=training_args,
    train_dataset=lm_corpus["train"],
    eval_dataset=lm_corpus["val"],
    data_collator=corpus_mask_collator,
    tokenizer=tokenizer,
)

/Users/chaosarium/anaconda3/envs/multi/lib/python3.10/site-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [9]:
trainer.evaluate()

100%|██████████| 1/1 [00:00<00:00, 5229.81it/s]


{'eval_loss': 12.164128303527832,
 'eval_runtime': 0.2334,
 'eval_samples_per_second': 4.284,
 'eval_steps_per_second': 4.284}

In [10]:
trainer.train()

 10%|█         | 1/10 [00:09<01:28,  9.86s/it]

{'loss': 15.9559, 'grad_norm': 483.2739562988281, 'learning_rate': 0.00045000000000000004, 'epoch': 1.0}


                                              
 10%|█         | 1/10 [00:10<01:28,  9.86s/it]

{'eval_loss': 11.50549030303955, 'eval_runtime': 0.2249, 'eval_samples_per_second': 4.446, 'eval_steps_per_second': 4.446, 'epoch': 1.0}


 20%|██        | 2/10 [00:20<01:22, 10.35s/it]

...

 30%|███       | 3/10 [00:31<01:13, 10.49s/it]

{'loss': 7.7883, 'grad_norm': 95.55084228515625, 'learning_rate': 0.00035, 'epoch': 3.0}


                                              
 30%|███       | 3/10 [00:31<01:13, 10.49s/it]

{'eval_loss': 8.914809226989746, 'eval_runtime': 0.3171, 'eval_samples_per_second': 3.153, 'eval_steps_per_second': 3.153, 'epoch': 3.0}


KeyboardInterrupt: 